In [2]:
!tar -xvzf data/data.tar.gz

data/
data/eval.py
data/cran.qry
data/qrel_clean
data/cran.all.1400


In [4]:
!head -30 data/cran.all.1400

.I 1
.T
experimental investigation of the aerodynamics of a
wing in a slipstream .
.A
brenckman,m.
.B
j. ae. scs. 25, 1958, 324.
.W
experimental investigation of the aerodynamics of a
wing in a slipstream .
  an experimental study of a wing in a propeller slipstream was
made in order to determine the spanwise distribution of the lift
increase due to slipstream at different angles of attack of the wing
and at different free stream to slipstream velocity ratios .  the
results were intended in part as an evaluation basis for different
theoretical treatments of this problem .
  the comparative span loading curves, together with
supporting evidence, showed that a substantial part of the lift increment
produced by the slipstream was due to a /destalling/ or
boundary-layer-control effect .  the integrated remaining lift
increment, after subtracting this destalling lift, was found to agree
well with a potential flow theory .
  an empirical evaluation of the destalling effects was made for
the 

In [8]:
import os
import sys

from typing import (
    Optional,
    List
)

BASE_DIR = "data"
ALL_DATA_FILE = os.path.join(BASE_DIR, "cran.all.1400")

In [10]:
class Text:
    def __init__(self):
        self.i = None # type: str
        self.t = None # type: str
        self.a = None # type: str
        self.b = None # type: str
        self.w = None # type: str


def read_file(filepath):
    # type: (str) -> Optional[List[str]]
    
    with open(filepath, "r") as fin:
        lines = fin.readlines()
        return lines
    
    return None


def parse_text(lines):
    # type: (str) -> List[Text]
    
    d = {
        1: 'i',
        2: 't',
        3: 'a',
        4: 'b',
        5: 'w',
    }
    
    texts = []
    
    while True:
        t = Text()
        for l in lines:
            if l.startswith(".I"):
                texts.append(line)
            elif l.startswith(".T"):
                text



['.Iasdf.Iasdf']

In [15]:
".Iasdf.Iasdf".split(".I")

['', 'asdf', 'asdf']